In [14]:
# target parameter attack on linear regression with close form solution on the cross derivative

import os
import time
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from models import *
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from numpy import linalg as LA
import numpy as np
import math

torch.manual_seed(0)
device = 'cuda'

In [15]:
num_inputs = 1
num_outputs = 1
num_examples_train = 10000
num_examples_test = 5000
dtype = torch.float

def real_fn(X):
    return 2 * X + 4.2
    #return 2 * X[:, 0] - 3.4 * X[:, 1] + 4.2

# define training set
X_train = torch.randn(num_examples_train, num_inputs, device=device, dtype=dtype)
noise_train = .1 * torch.randn(num_examples_train, num_inputs, device=device, dtype=dtype)
y_train = (real_fn(X_train) + noise_train)

X_test = torch.randn(num_examples_test, num_inputs, device=device, dtype=dtype)
noise_test = .1 * torch.randn(num_examples_test, num_inputs, device=device, dtype=dtype)
y_test = (real_fn(X_test) + noise_test)

In [16]:
class LinearDataset(Dataset):
    def __init__(self, X, y):
        assert X.size()[0] == y.size()[0]
        self.X = X
        self.y = y
    
    def __len__(self):
        return self.X.size()[0]
    
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [17]:
batch_size = 10000
train_data = DataLoader(LinearDataset(X_train, y_train), batch_size=batch_size, shuffle=False)
test_data = DataLoader(LinearDataset(X_test, y_test), batch_size=batch_size, shuffle=True)

In [18]:
class LinearRegression(nn.Module):
    def __init__(self, **kwargs):
        super(LinearRegression, self).__init__(**kwargs)
        self.fc1 = nn.Linear(1, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        return x
    
model = LinearRegression()
model.to(device)
model.load_state_dict(torch.load("models/linear_gd.pt"))

<All keys matched successfully>

In [185]:
# the script for training target attack
epsilon = 1e-4
lr = 0.05
epochs = 200
decay_rate = 0.96
decay_steps = 10000

def adjust_learning_rate(lr, epoch):
    """Decay the learning rate based on schedule"""
    lr *= 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    return(lr)


def autograd(outputs, inputs, create_graph=False):
    """Compute gradient of outputs w.r.t. inputs, assuming outputs is a scalar."""
    #inputs = tuple(inputs)
    grads = torch.autograd.grad(outputs, inputs, create_graph=create_graph, allow_unused=True)
    return [xx if xx is not None else yy.new_zeros(yy.size()) for xx, yy in zip(grads, inputs)]

def train(epoch):
    for batch_idx, (data, target) in enumerate(train_data):
        lr1 = adjust_learning_rate(lr,epoch)
        data, target = data.to(device), target.to(device)
        data.requires_grad=True
        if epoch==0:
            # initialize poisoned data
            data_p = Variable(data[:(int(epsilon*len(data)))])
            target_p = Variable(target[:(int(epsilon*len(target)))])
            max_value = torch.max(data_p)
            min_value = torch.min(data_p)
            torch.save(target_p,'target_p_linear.pt')
        else:
            data_p = torch.load('data_p_linear.pt')
            target_p = torch.load('target_p_linear.pt')
            max_value = torch.max(data_p)
            min_value = torch.min(data_p)
        data_p.requires_grad=True
        
    
        # initialize f function
        criterion = nn.MSELoss(reduction='sum')
        
        # calculate gradient of w on clean sample
        output_c = model(data.view(data.size(0), -1))
        loss_c =  0.5 * criterion(output_c,target)
        
        # calculate dL/dg_1
        grad_c = autograd(loss_c,tuple(model.parameters()),create_graph=True)
        #g1 = torch.cat((grad_c[0],grad_c[1].unsqueeze(0)),0)
        g1 = grad_c[0]
        
        # calculate gradient of w on poisoned sample
        output_p = model(data_p.view(data_p.size(0), -1))
        loss_p = 0.5 * criterion(output_p,target_p)
        grad_p= autograd(loss_p,tuple(model.parameters()),create_graph=True)
        #g2 = torch.cat((grad_p[0],grad_p[1].unsqueeze(0)),0)
        #g2 = torch.matmul((output_p - target_p).t(),data_p)
        g2 = grad_p[0]
        
        # calculate the true loss: |g_c + g_p|_{inf}
        grad_sum = g1+g2
        
        loss = torch.norm(grad_sum,2)
        if loss < 0.1:
            break
        
        update = autograd(loss,data_p,create_graph=True)
    
        data_t = data_p - lr1 *update[0]
        torch.save(data_t, 'data_p_linear.pt')
        
        print("epoch:{},lr:{},loss:{}".format(epoch, lr1,loss))
        

In [186]:
for epoch in range(epochs):
    train(epoch)

epoch:0,lr:0.05,loss:4500.52587890625
epoch:1,lr:0.04999691581204152,loss:4500.52490234375
epoch:2,lr:0.04998766400914329,loss:4500.5234375
epoch:3,lr:0.049972246874049255,loss:4500.52197265625
epoch:4,lr:0.04995066821070679,loss:4500.51953125
epoch:5,lr:0.0499229333433282,loss:4500.51611328125
epoch:6,lr:0.049889049115077,loss:4500.51220703125
epoch:7,lr:0.0498490238863795,loss:4500.50634765625
epoch:8,lr:0.04980286753286195,loss:4500.4990234375
epoch:9,lr:0.04975059144291394,loss:4500.48876953125
epoch:10,lr:0.04969220851487845,loss:4500.47509765625
epoch:11,lr:0.04962773315386935,loss:4500.45751953125
epoch:12,lr:0.049557181268217225,loss:4500.43359375
epoch:13,lr:0.049480570265544144,loss:4500.40185546875
epoch:14,lr:0.049397919048468686,loss:4500.359375
epoch:15,lr:0.049309248009941915,loss:4500.30322265625
epoch:16,lr:0.04921457902821578,loss:4500.22900390625
epoch:17,lr:0.049113935461444956,loss:4500.1298828125
epoch:18,lr:0.04900734214192358,loss:4499.998046875
epoch:19,lr:0.04

In [262]:
b = torch.ones(int(num_examples_train*epsilon))
b = b.unsqueeze(1)
b = b.to('cuda')
# absorb b into the input so it matches the weights in tensors
X_p_b = torch.cat((data_p,b),1)
Xw_input = torch.cat((X_p_b,w.t()),0)

In [ ]:
dldg2 = torch.autograd.grad(loss,g2,create_graph=True)
        #print(dldg2[0][0])
        
        #dldg2 = torch.stack(list(dldg2),dim=0)
        #dldg2 = dldg2.squeeze(0)
        
        # first approach is to calculate the update in closed form 
        l = []
        for param in model.parameters():
            l.append(param)
        # extract w and b
        w = torch.cat((l[0],l[1].unsqueeze(0)))
        
        
        update = torch.matmul(data_p,l[0]*dldg2[0][0])
        identity = torch.ones(len(data_p),1)
        identity = identity.to('cuda')
        #update1 = - torch.matmul((output_p - target_p)*identity,dldg2)
        update1 = -(output_p - target_p)*identity * dldg2[0][0]

        #print("approach 1 update:{}".format(update1))
        
        # Still bugs in Approach 2 and 3, needs to be fixed
        # second approach is to use torch.autograd.functional.hessian to calculate the update
        Xw_input = torch.cat((data_p,w[0].unsqueeze(0).t()),0)
        def function_f(x):
            y_hat = torch.matmul(x[:(len(x)-1)],x[(len(x)-1):].t())+w[1]
            return(0.5 * criterion(y_hat,target_p))
        f_x = function_f(Xw_input)
        hessian = torch.autograd.functional.hessian(function_f, Xw_input)
        hessian_wx = hessian[len(Xw_input)-1]
        hessian_wx = hessian_wx.squeeze(0)        
            
        #update2 = torch.matmul(hessian_wx,dldg2[0]).unsqueeze(1)
        #print("approach2 update:{}".format(update2))
        
        # third approach is to use torch.autograd.functional.hvp to calculate the update
        v = torch.ones(len(data_p)+1,1) 
        v = v.to('cuda')
        #v = v * dldg2[0]
        v[len(v)-1] = 0
        update3 = torch.autograd.functional.hvp(function_f,Xw_input,v)
        #print(update3[1])

In [ ]:
# random tests, may be useful
criterion = nn.MSELoss()
a = torch.ones(3)
inputs = torch.ones(10, 1)
def function_f(x):
    y_hat = torch.matmul(x[:(len(x)-1)],x[(len(x)-1):].t())
    return(criterion(y_hat,a))
function_f(inputs)
print(torch.autograd.functional.hessian(function_f, inputs)[2].size())

In [67]:
def pow_reducer(x):
    return x.pow(3).sum()
    
inputs1 = torch.rand(2,2)
inputs2 = torch.rand(1,2)
print(6*inputs1)
print(6*inputs2)
#inputs2 = inputs1[0]
#a = torch.ones(1,2)
#b = torch.zeros(2,2)
#v = torch.cat((b,a.t()),0)
print(pow_reducer(inputs1))
print(torch.autograd.functional.hessian(pow_reducer, inputs1))
print(torch.autograd.functional.hessian(pow_reducer, inputs2))
#torch.autograd.functional.hvp(pow_reducer, inputs,v)

tensor([[0.8822, 1.5141],
        [0.5289, 4.5655]])
tensor([[2.6943, 5.3088]])
tensor(0.4605)
tensor([[[[0.8822, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 1.5141],
          [0.0000, 0.0000]]],


        [[[0.0000, 0.0000],
          [0.5289, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 4.5655]]]])
tensor([[[[2.6943, 0.0000]],

         [[0.0000, 5.3088]]]])


In [ ]:
l = []
for param in model.parameters():
    l.append(param)

# extract w and b
w = torch.cat((l[0],l[1].unsqueeze(0)),0)
print(w.t().size())

b = torch.ones(num_examples_train)
b = b.unsqueeze(1)
b = b.to('cuda')

# absorb b into weights
X_train_b = torch.cat((X_train,b),1)
print(X_train_b.size())

# retrieve prediction using matrix multiplication
y_hat = torch.matmul(X_train_b,w)
print(y_hat)

In [263]:
l = []
for param in model.parameters():
    l.append(param)

# extract w and b
w = torch.cat((l[0],l[1].unsqueeze(0)),0)
print(w.t().size())

b = torch.ones(num_examples_train)
b = b.unsqueeze(1)
b = b.to('cuda')

# absorb b into weights
X_train_b = torch.cat((X_train,b),1)
print(X_train_b.size())

# retrieve prediction using matrix multiplication
y_hat = torch.matmul(X_train_b,w)
print(y_hat)

torch.Size([1, 2])
torch.Size([10000, 2])
tensor([[3.6727],
        [4.4504],
        [0.9949],
        ...,
        [5.0574],
        [5.3945],
        [5.1862]], device='cuda:0', grad_fn=<MmBackward>)
